In [27]:
from transformers import AutoModelForCausalLM
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset

device = "cuda"
model_name_or_path = "bigscience/bloomz-560m"
tokenizer_name_or_path = "bigscience/bloomz-560m"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="Classify if the sentence is positive or negative:",
    tokenizer_name_or_path=model_name_or_path,
)

dataset_name = "sst2"
checkpoint_name = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)
text_column = "sentence"
label_column = "text_label"
max_length = 64
lr = 3e-2
num_epochs = 1
batch_size = 8

In [2]:
from datasets import load_dataset

dataset = load_dataset("glue", dataset_name)

Found cached dataset glue (/mnt/nlp4sd/hugging_face_cache/ysong/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [4]:
classes = [k for k in dataset["train"].features["label"].names]
print(classes)
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)
print(dataset)
dataset["train"][0]

Loading cached processed dataset at /mnt/nlp4sd/hugging_face_cache/ysong/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-580ea0d570b9d184.arrow
Loading cached processed dataset at /mnt/nlp4sd/hugging_face_cache/ysong/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-881202904f0b1837.arrow
Loading cached processed dataset at /mnt/nlp4sd/hugging_face_cache/ysong/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-6cc85fa0c11cfa8a.arrow


['negative', 'positive']
DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'text_label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'text_label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'text_label'],
        num_rows: 1821
    })
})


{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0,
 'text_label': 'negative'}

In [5]:
# data preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
print(target_max_length)

1


In [6]:
def preprocess_function(examples):
    text_column = "sentence"
    label_column = "text_label"
    batch_size = len(examples[text_column])
    
    # Embed the labels into the prompt
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]

    # First tokenize the inputs
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)

    # Concatenate the inputs and labels and set -100 for labels
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        # Why Add eos_token_id at the end of the label for showing end?
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        
        # Concatenate the inputs and labels and set -100 for labels
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        # Set all attention mask to 1
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    
    # Pad the inputs and labels
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids

        # Truncate the inputs and labels
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/67349 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/872 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [29]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

from torch.utils.data import random_split

torch.manual_seed(42)
train_len = len(train_dataset)
half_len = int(train_len * 0.1)
print(half_len)

subset_train_dataset, _ = random_split(train_dataset, [half_len, train_len - half_len])


train_dataloader = DataLoader(
    subset_train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

6734


In [12]:
def test_preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    model_inputs = tokenizer(inputs)
    targets = [str(x) for x in examples[label_column]]
    labels = tokenizer(targets)

    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


test_dataset = dataset["validation"].map(
    test_preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

test_dataloader = DataLoader(test_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

Running tokenizer on dataset:   0%|          | 0/872 [00:00<?, ? examples/s]

In [21]:
# creating model
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 8192 || all params: 559222784 || trainable%: 0.0014648902430985358


In [22]:
# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [30]:
# training and evaluation
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch}=: {train_ppl}= {train_epoch_loss}= {eval_ppl}= {eval_epoch_loss}=")

100%|██████████| 109/109 [00:12<00:00,  8.62it/s]

epoch=0: train_ppl=tensor(1.1649, device='cuda:0') train_epoch_loss=tensor(0.1526, device='cuda:0') eval_ppl=tensor(1.1442, device='cuda:0') eval_epoch_loss=tensor(0.1347, device='cuda:0')


In [38]:
model.eval()
test_preds = []
labels = []

for step, batch in enumerate(tqdm(test_dataloader)):
    batch = {k: v.to(device) for k, v in batch.items()}
    ids_preds = []

    with torch.no_grad():
        outputs = model.generate(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], max_new_tokens=10, eos_token_id=3
        )
    
    # print(tokenizer.batch_decode(outputs[:, -2:-1]))
    # print(tokenizer.batch_decode(batch["labels"]))
    test_preds.extend(tokenizer.batch_decode(outputs[:, -2:-1]))
    labels.extend(tokenizer.batch_decode(batch["labels"]))
   


100%|██████████| 109/109 [00:15<00:00,  7.21it/s]


In [48]:
def get_metrics(preds, labels):
    return [int(pred == label) for pred, label in zip(preds, labels)]

In [49]:
def calculate_percentage(lst):
    total = len(lst)
    count_ones = sum(lst)
    percentage = count_ones / total * 100
    return percentage

In [50]:
res = get_metrics(test_preds, labels)
print(calculate_percentage(res))

92.08715596330275
